In [1]:
### no brand names and search terms vector

In [2]:
%load_ext Cython

In [7]:
%%time
%%cython
from gensim.models import doc2vec
import nltk
import re
def split_sentence (sentence):
    return re.split('\W+',sentence)
    
class MyDocs(object):
    def __iter__(self):
        for i, text in enumerate(open("txt_pt_st.txt")): # doesn't use brand name
            yield doc2vec.LabeledSentence(words=split_sentence(text), tags=['%s' % i])
            # Train the doc2vec model
cdef desp = MyDocs()
model = doc2vec.Doc2Vec(desp, size = 200, window = 8, min_count = 5, workers = 4)
model.save('distance_pt_st.model')

CPU times: user 439 µs, sys: 227 µs, total: 666 µs
Wall time: 516 µs


In [8]:
model = doc2vec.Doc2Vec.load('distance_pt_st.model')

In [9]:
# Calculate the vector of search term according to our model
#def input_vec(x):
#    model.infer_vector(split_sentence(x))
# map(input_vec,open("txt_st.txt))
import numpy as np
pt_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_pt.txt")): # input search terms 
    pt_arrays[i]=model.infer_vector(split_sentence(text))   
pt_arrays

array([[-0.00956058, -0.0218237 ,  0.02176641, ..., -0.01747051,
        -0.01045967, -0.00127221],
       [ 0.02217622, -0.04077116,  0.04018299, ..., -0.01987829,
        -0.03357719,  0.00353559],
       [ 0.07837269, -0.01117249,  0.0186554 , ..., -0.00659222,
        -0.06981758,  0.01006482],
       ..., 
       [-0.00630262,  0.00282859, -0.00563839, ...,  0.06102261,
         0.03075218,  0.01499327],
       [-0.00592348, -0.04118564,  0.02782599, ..., -0.00174693,
         0.00603268, -0.03693224],
       [ 0.00593108, -0.01655589, -0.01038504, ...,  0.02085312,
        -0.03878854, -0.00564496]])

In [10]:
import numpy as np
st_arrays = np.zeros((240760, 200))
for i,text in enumerate(open("txt_st.txt")): # input search terms 
    st_arrays[i]=model.infer_vector(split_sentence(text))   
st_arrays

array([[ 0.08624037, -0.01894077,  0.04079454, ..., -0.00332129,
        -0.05674257, -0.0428574 ],
       [ 0.0655307 , -0.02469077,  0.04190942, ...,  0.02236441,
        -0.03334444,  0.00738343],
       [-0.06238248, -0.00161002, -0.06827073, ...,  0.03149903,
        -0.04201627,  0.03962637],
       ..., 
       [ 0.09644369,  0.00639722,  0.10442176, ...,  0.03069219,
         0.03578154, -0.00592293],
       [ 0.08057009, -0.03957823,  0.02823567, ...,  0.02709434,
        -0.05115322, -0.01225009],
       [ 0.06302886, -0.00649855,  0.00910959, ...,  0.02563455,
        -0.00201748, -0.00352245]])

In [11]:
## calculate the cosin distance between 2 vectors to generate the 3rd features
# Compute the cosine similarity values between the input text and all archived reviews
# cossims_with_input = map(lambda v: cossim(input_vec, v), model.docvecs) 
# need to chaneg the code into rows: calculate the cosin distance between the vectors in same rows
import numpy as np
import pandas as pd
# Calculate the cosine similarity between two vecotrs 
def cossim(v1, v2):
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2))

cossims_with_st=np.zeros((240760,1))
for i in range(240760):
    cossims_with_st[i] = cossim(st_arrays[i], pt_arrays[i])

In [13]:
np.savetxt("distance_pt_st.csv",cossims_with_st, delimiter=",")

In [14]:
coswithst=pd.read_csv("distance_pt_st.csv",header=None)
coswithst


,0
0,0.116122
1,0.195325
2,0.240977
3,0.684551
4,0.429076
5,0.437666
6,0.297869
7,0.437242
8,0.224700
9,0.112295


In [ ]:
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]